In [1]:
import numpy as np
import pandas as pd
import time
import tempfile
import os
from pathlib import Path

# DICOM libraries
import pydicom
import dicube

# NIfTI libraries  
import nibabel as nib

os.chdir("../")

print("✅ All libraries imported successfully")



✅ All libraries imported successfully


In [2]:
def load_dicom_with_pydicom(dicom_folder):
    """Load DICOM files using pydicom and return pixel arrays"""
    dicom_files = sorted([f for f in Path(dicom_folder).glob("*.dcm")])
    datasets = []
    pixel_arrays = []
    
    for dcm_file in dicom_files:
        ds = pydicom.dcmread(dcm_file)
        datasets.append(ds)
        pixel_arrays.append(ds.pixel_array)
    
    return datasets, np.array(pixel_arrays)

def save_dicom_with_pydicom(datasets, pixel_arrays, output_folder):
    """Save DICOM files using pydicom"""
    os.makedirs(output_folder, exist_ok=True)
    
    for i, (ds, pixel_array) in enumerate(zip(datasets, pixel_arrays)):
        # Update pixel data
        ds.PixelData = pixel_array.tobytes()
        
        # Save to new location
        output_path = Path(output_folder) / f"slice_{i:04d}.dcm"
        ds.save_as(output_path)

print("✅ PyDICOM helper functions defined")


✅ PyDICOM helper functions defined


In [3]:
# Test data path
dicom_folder = "testdata/dicom/sample_150"

if not os.path.exists(dicom_folder):
    print(f"❌ DICOM folder not found: {dicom_folder}")
else:
    print(f"✅ DICOM folder found: {dicom_folder}")
    
    results = []
    
    with tempfile.TemporaryDirectory() as tmpdir:
        tmpdir = Path(tmpdir)
        
        # ======= PyDICOM Method =======
        print("\n🔄 Testing PyDICOM...")
        
        # Load with pydicom
        start_time = time.time()
        datasets, pixel_arrays = load_dicom_with_pydicom(dicom_folder)
        pydicom_load_time = time.time() - start_time
        
        print(f"  Loaded {len(datasets)} DICOM files")
        print(f"  Shape: {pixel_arrays.shape}, dtype: {pixel_arrays.dtype}")
        print(f"  Load time: {pydicom_load_time:.3f}s")
        
        # Save with pydicom
        pydicom_output = tmpdir / "pydicom_output"
        start_time = time.time()
        save_dicom_with_pydicom(datasets, pixel_arrays, pydicom_output)
        pydicom_save_time = time.time() - start_time
        
        # Calculate folder size
        pydicom_size = sum(f.stat().st_size for f in pydicom_output.glob("*.dcm"))
        
        print(f"  Save time: {pydicom_save_time:.3f}s")
        print(f"  Folder size: {pydicom_size / 1024 / 1024:.2f} MB")
        
        results.append({
            'Method': 'PyDICOM',
            'Load Time (s)': pydicom_load_time,
            'Save Time (s)': pydicom_save_time,
            'File Size (MB)': pydicom_size / 1024 / 1024,
            'Compression Ratio': pixel_arrays.nbytes / pydicom_size
        })
        
        # ======= DicomCubeImage Method =======
        print("\n🔄 Testing DicomCubeImage...")
        
        # Load with DicomCubeImage
        start_time = time.time()
        dicube_image = dicube.load_from_dicom_folder(dicom_folder)
        dicube_load_time = time.time() - start_time
        
        print(f"  Shape: {dicube_image.shape}, dtype: {dicube_image.raw_image.dtype}")
        print(f"  Load time: {dicube_load_time:.3f}s")
        
        # Save as DCB file
        dcb_file = tmpdir / "test.dcbs"
        start_time = time.time()
        dicube.save(dicube_image, str(dcb_file), file_type="s")  # "s" for speed (OJPH)
        dicube_save_time = time.time() - start_time
        
        # Load DCB file
        start_time = time.time()
        loaded_image = dicube.load(str(dcb_file))
        dcb_load_time = time.time() - start_time
        
        dcb_size = dcb_file.stat().st_size
        
        print(f"  DCB save time: {dicube_save_time:.3f}s")
        print(f"  DCB load time: {dcb_load_time:.3f}s")
        print(f"  DCB file size: {dcb_size / 1024 / 1024:.2f} MB")
        
        # Verify data consistency
        data_consistent = np.array_equal(dicube_image.raw_image, loaded_image.raw_image)
        print(f"  Data consistent: {data_consistent}")
        
        results.append({
            'Method': 'DCB (OJPH)',
            'Load Time (s)': dcb_load_time,
            'Save Time (s)': dicube_save_time,
            'File Size (MB)': dcb_size / 1024 / 1024,
            'Compression Ratio': pixel_arrays.nbytes / dcb_size
        })
    
    # Display comparison table
    df = pd.DataFrame(results)
    df = df.round(3)
    print("\n📊 DICOM Comparison Results:")
    print(df.to_string(index=False))


✅ DICOM folder found: testdata/dicom/sample_150

🔄 Testing PyDICOM...
  Loaded 150 DICOM files
  Shape: (150, 512, 512), dtype: uint16
  Load time: 0.385s
  Save time: 0.268s
  Folder size: 75.22 MB

🔄 Testing DicomCubeImage...
  Shape: (150, 512, 512), dtype: uint16
  Load time: 0.763s
  DCB save time: 0.187s
  DCB load time: 0.141s
  DCB file size: 36.27 MB
  Data consistent: True

📊 DICOM Comparison Results:
    Method  Load Time (s)  Save Time (s)  File Size (MB)  Compression Ratio
   PyDICOM          0.385          0.268          75.223              0.997
DCB (OJPH)          0.141          0.187          36.265              2.068


In [4]:
def load_nifti_with_nibabel(nifti_file):
    """Load NIfTI file using nibabel"""
    nii = nib.load(nifti_file)
    data = np.asarray(nii.dataobj, dtype=nii.dataobj.dtype)
    return nii, data

def save_nifti_with_nibabel(data, affine, output_file):
    """Save NIfTI file using nibabel"""
    nii = nib.Nifti1Image(data, affine)
    nib.save(nii, output_file)

print("✅ NiBabel helper functions defined")


✅ NiBabel helper functions defined


In [5]:
# Test data path
nifti_file = "testdata/nifti/s0000.nii.gz"

if not os.path.exists(nifti_file):
    print(f"❌ NIfTI file not found: {nifti_file}")
else:
    print(f"✅ NIfTI file found: {nifti_file}")
    
    results = []
    original_size = os.path.getsize(nifti_file)
    print(f"  Original file size: {original_size / 1024 / 1024:.2f} MB")
    
    with tempfile.TemporaryDirectory() as tmpdir:
        tmpdir = Path(tmpdir)
        
        # ======= NiBabel Method =======
        print("\n🔄 Testing NiBabel...")
        
        # Load with nibabel
        start_time = time.time()
        nii, nifti_data = load_nifti_with_nibabel(nifti_file)
        nibabel_load_time = time.time() - start_time
        
        print(f"  Shape: {nifti_data.shape}, dtype: {nifti_data.dtype}")
        print(f"  Load time: {nibabel_load_time:.3f}s")
        
        # Save with nibabel
        nibabel_output = tmpdir / "nibabel_output.nii.gz"
        start_time = time.time()
        save_nifti_with_nibabel(nifti_data, nii.affine, nibabel_output)
        nibabel_save_time = time.time() - start_time
        
        nibabel_size = nibabel_output.stat().st_size
        
        print(f"  Save time: {nibabel_save_time:.3f}s")
        print(f"  File size: {nibabel_size / 1024 / 1024:.2f} MB")
        
        results.append({
            'Method': 'NiBabel',
            'Load Time (s)': nibabel_load_time,
            'Save Time (s)': nibabel_save_time,
            'File Size (MB)': nibabel_size / 1024 / 1024,
            'Compression Ratio': nifti_data.nbytes / nibabel_size
        })
        
        # ======= DicomCubeImage Method =======
        print("\n🔄 Testing DicomCubeImage...")
        
        # Load with DicomCubeImage
        start_time = time.time()
        dicube_image = dicube.load_from_nifti(nifti_file)
        dicube_load_time = time.time() - start_time
        
        print(f"  Shape: {dicube_image.shape}, dtype: {dicube_image.raw_image.dtype}")
        print(f"  Load time: {dicube_load_time:.3f}s")
        
        # Save as DCB file
        dcb_file = tmpdir / "test_nifti.dcbs"
        start_time = time.time()
        dicube.save(dicube_image, str(dcb_file), file_type="s")  # "s" for speed (OJPH)
        dicube_save_time = time.time() - start_time
        
        # Load DCB file
        start_time = time.time()
        loaded_image = dicube.load(str(dcb_file))
        dcb_load_time = time.time() - start_time
        
        dcb_size = dcb_file.stat().st_size
        
        print(f"  DCB save time: {dicube_save_time:.3f}s")
        print(f"  DCB load time: {dcb_load_time:.3f}s")
        print(f"  DCB file size: {dcb_size / 1024 / 1024:.2f} MB")
        
        # Verify data consistency
        data_consistent = np.array_equal(dicube_image.raw_image, loaded_image.raw_image)
        print(f"  Data consistent: {data_consistent}")
        
        results.append({
            'Method': 'DCB (OJPH)',
            'Load Time (s)': dcb_load_time,
            'Save Time (s)': dicube_save_time,
            'File Size (MB)': dcb_size / 1024 / 1024,
            'Compression Ratio': nifti_data.nbytes / dcb_size
        })
        
    
    # Display comparison table
    df = pd.DataFrame(results)
    df = df.round(3)
    print("\n📊 NIfTI Comparison Results:")
    print(df.to_string(index=False))


✅ NIfTI file found: testdata/nifti/s0000.nii.gz
  Original file size: 10.99 MB

🔄 Testing NiBabel...
  Shape: (294, 192, 179), dtype: int16
  Load time: 0.145s
  Save time: 0.316s
  File size: 10.99 MB

🔄 Testing DicomCubeImage...
  Shape: (294, 192, 179), dtype: uint16
  Load time: 0.142s
  DCB save time: 0.072s
  DCB load time: 0.050s
  DCB file size: 5.98 MB
  Data consistent: True

📊 NIfTI Comparison Results:
    Method  Load Time (s)  Save Time (s)  File Size (MB)  Compression Ratio
   NiBabel          0.145          0.316          10.993              1.753
DCB (OJPH)          0.050          0.072           5.979              3.223
